In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.svm import SVC

from math import nan

from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline 

pd.set_option('display.max_columns', None)

In [ ]:
path = '../data/processed/combined_data.csv'
df = pd.read_csv(path, index_col='date')


In [ ]:
df

In [ ]:
y = df.iloc[:,2]
mask_f = ~df.loc[:,'Ferrero.1'].isna()
#pd.DataFrame(y).loc[mask_f ,:]
y.loc[mask_f]


In [ ]:
from sklearn.linear_model import LinearRegression


class Model(TransformerMixin, BaseEstimator):
    """Imputs missing values. """
    
    def __init__(self,columns):# model1, model2, model3, 
        """Initalizes the attributes."""
        self.features = []
        self.num_cols = []
        self.dict_num = {}
        self.model_start = LinearRegression()
        self.model_middle = LinearRegression()#model2
        self.model_end = LinearRegression()#model3
        self.columns = columns

    def fit(self, X, y=None):
        mask_f = X.loc[:,'Ferrero.1'].isna()
        mask_l = X.loc[:,'LindtSpruengli.1'].isna()

        feat_start = X#.loc[:, self.columns + ['Mondelez.1']]
        target_start = y
        self.model_start.fit(feat_start.loc[:, self.columns + ['Mondelez.1']], target_start)

        feat_middle = X.copy()
        feat_middle = feat_middle.loc[~mask_f , :]
        target_middle = y.loc[~mask_f]
        feat_middle['target_pred_by_1'] = self.model_start.predict(feat_middle.loc[:, self.columns + ['Mondelez.1']])

        self.model_middle.fit( feat_middle.loc[:,['target_pred_by_1', 'Ferrero.1']],target_middle)

        feat_end = X.copy()
        feat_end = feat_end.loc[~mask_f & ~mask_l, :]
        target_end = y.loc[~mask_f & ~mask_l]
        feat_end['target_pred_by_1'] = self.model_start.predict(feat_end.loc[:, self.columns + ['Mondelez.1']])
        feat_end['target_pred_by_2'] = self.model_middle.predict(feat_end.loc[:,['target_pred_by_1', 'Ferrero.1']])

        self.model_end.fit(feat_end.loc[:,['target_pred_by_2', 'LindtSpruengli.1']],target_end)

        #self.model_start = self.model_start
        #self.model_middle = self.model_middle
        #self.model_end = self.model_end
        #print('almost')
        return self
    
    def transform(self, X, y=None):
        """Fill the missing values and drop some columns.
        
        
        """
        df = X.copy()

        self.features = df.columns.values
        return df
    
    def predict(self, X, y=None):
        """Fill the missing values and drop some columns.
        
        
        """
        mask_f = X.loc[:,'Ferrero.1'].isna()
        mask_l = X.loc[:,'LindtSpruengli.1'].isna()

        #feat_start = X#.loc[:, self.columns + ['Mondelez.1']]
        #target_start = y
        #model_start.fit(feat_start.loc[:, self.columns + ['Mondelez.1']], target_start)
        result = pd.DataFrame.from_dict({'index': X.index.values})
        result = result.set_index('index')
        result['pred_m']=nan
        result['pred_mf']=nan
        result['pred_mfl']=nan
        feat_middle = X.copy()
        feat_middle = feat_middle.loc[~mask_f , :]
        target_middle = y.loc[~mask_f]
        feat_middle['target_pred_by_1'] = self.model_start.predict(feat_middle.loc[:, self.columns + ['Mondelez.1']])
        result.loc[: ,'pred_m']= self.model_start.predict(X.loc[:, self.columns + ['Mondelez.1']])
        #model_middle.fit( feat_middle.loc[:,['target_pred_by_1', 'Ferrero.1']],target_middle)

        feat_end = X.copy()
        feat_end = feat_end.loc[~mask_f & ~mask_l, :]
        target_end = y.loc[~mask_f & ~mask_l]
        feat_end['target_pred_by_1'] = self.model_start.predict(feat_end.loc[:, self.columns + ['Mondelez.1']])
        feat_end['target_pred_by_2'] = self.model_middle.predict(feat_end.loc[:,['target_pred_by_1', 'Ferrero.1']])
        result.loc[~mask_f ,'pred_mf']= self.model_middle.predict(feat_middle.loc[:, ['target_pred_by_1', 'Ferrero.1']])
        result.loc[~mask_f & ~mask_l ,'pred_mfl']= self.model_end.predict(feat_end.loc[:,  ['target_pred_by_2', 'LindtSpruengli.1']])


        return result
    
    def get_feature_names_out(self, input_features=None):
        """Returns the names of the features."""
        return self.features

In [ ]:
target_name = 'Wert der zum Absatz bestimmten Produktion_Schokolade u.a. kakaohaltige Lebensmittelzubereit.'
tar_name = df[target_name]
number_train = 74
shares = ['Ferrero.1', 'LindtSpruengli.1', 'Mondelez.1']
df_selected_col = df.loc[:, shares + [target_name, 'ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg']]
df_train = df_selected_col.iloc[:number_train, :]
df_test = df_selected_col.iloc[number_train: , :]

In [ ]:
target_train = df_train[target_name]
target_test = df_test[target_name]
features_train = df_train.loc[:, shares + ['ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg']]
features_test = df_test.loc[:, shares + ['ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg']]

In [ ]:
features_train

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
mod = Model( ['ErzPr_Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg'])
#LinearRegression(),LinearRegression(), LinearRegression()


In [ ]:
mod.fit(features_train, target_train)

In [ ]:
results = mod.predict(features_test, target_test)
results

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,5))
ax.plot(results.index,results.pred_m.values, label = 'm');
ax.plot(results.index,results.pred_mf.values, label = 'mf');
ax.plot(results.index,results.pred_mfl.values, label = 'mfl');
ax.plot(results.index,target_test, color =  'red', label = 'target');
ax.legend()

In [ ]:
results = mod.predict(features_train, target_train)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,5))
ax.plot(results.index,results.pred_m.values, label = 'm');
ax.plot(results.index,results.pred_mf.values, label = 'mf');
ax.plot(results.index,results.pred_mfl.values, label = 'mfl');
ax.plot(results.index,target_train, color =  'red', label = 'target');
ax.legend()

In [ ]:
features_test

In [ ]:
target_test

In [ ]:
target = 'Wert der zum Absatz bestimmten Produktion_Schokolade u.a. kakaohaltige Lebensmittelzubereit.'
number_train = 70
# the dict has entry of the form column: [lag_0, lag_1,lag_2]
# note that the column must be given by the name and not the number and that if the 
# column should be kept it need to have lag = 0
dict_lag = {}
for k in [39, 41, 43, 45, 78, 88, 93]:
    dict_lag.update({df.columns.values[k]: [1,2,3]})
for k in [166, 167, 168, 169, 170]: 
    dict_lag.update({df.columns.values[k]: [0]})

df_selected_col = df.loc[:, list(dict_lag)]
df_train = df_selected_col.iloc[:number_train, :]
df_test = df_selected_col.iloc[number_train: , :]
df_train

pipeline = Pipeline([('DaIm', DataImputation()), 
                     ('FE', FeatureEngineer(dict_lag))])

df_train_transf = pipeline.fit_transform(df_train)
df_test_transf = pipeline.transform(df_test)
df_train_transf